Overview:
- create an instance of IBM Speech to Text, and IBM Language Translator. 
- IBM Speech to Text is a service that converts speech in audio format into text. 
- Watson Speech to Text is a service that uses deep learning algorithms to convert speech to text. 
- IBM Language Translator converts one language to another.  

Goals:  
- to convert an audio file of an English speaker to text using a Speech to Text API. 
- Then you will translate the English version to a Spanish version using a Language Translator API. 

In [1]:
# pip install ibm_watson wget

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import json
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [9]:
# url and my api key saved in config.py file
import config
url_s2t = "https://stream.watsonplatform.net/speech-to-text/api"

In [41]:
# create a Speech To Text Adapter object. the parameters are the endpoint and API key.
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

In [11]:
filename = 'PolynomialRegressionandPipelines.mp3'

In [8]:
with open(filename, mode='rb') as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

In [12]:
response.result

{'results': [{'alternatives': [{'confidence': 0.93,
     'transcript': 'in this video we will cover polynomial regression and pipelines '}],
   'final': True},
  {'alternatives': [{'confidence': 0.9,
     'transcript': "what do we do when a linear model is not the best fit for our data let's look into another type of regression model the polynomial regression we transform our data into a polynomial then use linear regression to fit the parameters that we will discuss pipelines pipelines are way to simplify your code "}],
   'final': True},
  {'alternatives': [{'confidence': 0.95,
     'transcript': "polynomial regression is a special case of the general linear regression this method is beneficial for describing curvilinear relationships what is a curvilinear relationship it's what you get by squaring or setting higher order terms of the predictor variables in the model transforming the data the model can be quadratic which means the predictor variable in the model is squared we use a b

In [15]:
from pandas.io.json import json_normalize
json_normalize(response.result['results'], 'alternatives')

C:\Users\anita\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,confidence,transcript
0,0.93,in this video we will cover polynomial regress...
1,0.90,what do we do when a linear model is not the b...
2,0.95,polynomial regression is a special case of the...
3,0.95,the model can be cubic which means the predict...
4,0.91,there also exists higher order polynomial regr...
5,0.89,let's look at an example from our data we gene...
6,0.92,in python we do this by using the poly fit fun...
7,0.90,negative one point five five seven X. one cute...
8,0.90,consider the feature shown here applying the m...
9,0.89,pipeline sequentially perform a series of tran...


In [32]:
content = response.result
recognized_text = content['results'][0]['alternatives'][0]['transcript']
recognized_text

'in this video we will cover polynomial regression and pipelines '

# Language Translator

In [33]:
from ibm_watson import LanguageTranslatorV3

In [34]:
url_lt = 'https://gateway.watsonplatform.net/language-translator/api'

In [35]:
version_lt='2018-05-01'

In [58]:
import config2
import config

In [59]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt, authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

In [60]:
# can get a Lists the languages that the service can identify. The method Returns the language code.
from pandas.io.json import json_normalize
json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

C:\Users\anita\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
...,...,...
70,uk,Ukrainian
71,ur,Urdu
72,vi,Vietnamese
73,zh,Simplified Chinese


Use the method translate this will translate the text. The parameter text is the text. Model_id is the type of model we would like to use use we use list the the langwich . In this case, we set it to 'en-zh' or English to Chinese. We get a Detailed Response object translation_response

In [61]:
# translate the text using method "translate". 
translation_response = language_translator.translate(\
                    text=recognized_text, model_id='en-zh')
translation_response

In [62]:
# result is a dictionary
translation = translation_response.get_result()
translation

{'translations': [{'translation': '在此视频中我们将覆盖多项式回归和管道 '}],
 'word_count': 10,
 'character_count': 64}

In [67]:
chinese_translation = translation['translations'][0]['translation']
chinese_translation

'在此视频中我们将覆盖多项式回归和管道 '

In [70]:
translation_response = language_translator.translate(\
                    text=chinese_translation, model_id='zh-en')
translation = translation_response.get_result()
translation

{'translations': [{'translation': 'In this video we will override the polynomial regression and pipeline '}],
 'word_count': 6,
 'character_count': 19}

In [74]:
english_translation = translation['translations'][0]['translation']
english_translation

'In this video we will override the polynomial regression and pipeline '

Translate this into French using original english text

In [106]:
translation_response = language_translator.translate(\
                    text=recognized_text, model_id='en-fr')
translation_response

In [107]:
translation = translation_response.get_result()
french_translation = translation['translations'][0]['translation']
french_translation

'Dans cette vidéo, nous couvrons la régression polynomiale et les pipelines '

In [121]:
def translate(langs):
    translation_response = language_translator.translate(
        text=recognized_text, model_id=langs)
    print('Original English text is: {}'.format(recognized_text))
    translation = translation_response.get_result()
    lang_translation = translation['translations'][0]['translation']
    print('Translation is: {}'.format(lang_translation))

In [122]:
translate('en-zh')

Original English text is: in this video we will cover polynomial regression and pipelines 
Translation is: 在此视频中我们将覆盖多项式回归和管道 
